In [1]:
import pymongo as pm
import mysql.connector as mdb
import requests as rq
import ujson as js
import pandas as pd
import numpy as np
from datetime import datetime as dt
from threading import Thread
from time import sleep

from utils.Container import Container
import _CONST as CONST

In [2]:
class Tracker:
    def __init__(self, char_id):
        self._urls = CONST.URLS
        self._args = CONST.ARGS
        self._data = CONST.SEED_DATA
        self._refresh = CONST.SEED_REFRESH
        self._threads = CONST.SEED_THREADS
        self._token = CONST.SEED_TOKEN
        
        self.char_id = char_id
        self.token_short_sleep_sec = CONST.TOKEN_SHORT_SLEEP_SEC
        self.url_retrys = CONST.URL_RETRYS
        
        self._build_conns(CONST.MONGO_LOGIN_PATH, CONST.MARIA_LOGIN_PATH)
        now_time = dt.utcnow()
        for key in self._urls: self._refresh[key] = now_time
            
    @property
    def args(self):
        args = self._args._dict
        args['headers']['Authorization'] = args['headers']['Authorization'].format(token=self._token.value)
        return args
    
    def start_threads(self):
        self._start_token_thread()
        self._start_data_threads()
        
    def _start_token_thread(self):
        self._threads.token = Thread(target=self._token_thread, daemon=True)
        self._threads.token.start()
            
    def _start_data_threads(self):
        self._threads.data = [
            Thread(target=self._data_thread, daemon=True, args=(data_key,))
            for data_key in self._urls
        ]
        [thread.start() for thread in self._threads.data]
    
    def _build_conns(self, mongo_login_path:str, maria_login_path:str):
        with open(mongo_login_path) as file_mongo, \
             open(maria_login_path) as file_maria:
            self.conn = Container(
                mongo=pm.MongoClient(**js.load(file_mongo)),
                maria=mdb.connect(**js.load(file_maria)),
                url=rq.Session()
            )
        
    def _token_thread(self):
        while True:
            self.refresh_token()
            sleep_delay = (self._token.expires - dt.now()).total_seconds() - self.token_short_sleep_sec
            if sleep_delay > 0: sleep(sleep_delay)
            
    def refresh_token(self):
        token_data = self.conn.mongo[self._token.db][self._token.coll].find_one(
            {'_id':self.char_id},
            {'tokens.access_token':1, 'tokens.expires_on':1}
        )['tokens']
        self._token.value = token_data['access_token']
        self._token.expires = token_data['expires_on']
        
    def _data_thread(self, data_key:str):
        while True:
            self.get_data(data_key)
            sleep_delay = (self._refresh[data_key] - dt.now()).total_seconds()
            if sleep_delay > 0: sleep(sleep_delay)
            
    def get_data(self, data_key:str):
        if dt.utcnow() > self._refresh[data_key]:
            resp = self._call_url('get', self._urls[data_key].format(char_id=self.char_id), self.args)
            if resp is not None:
                self._refresh[data_key] = dt.strptime(resp.headers['Expires'], '%a, %d %b %Y %H:%M:%S %Z')
                self._data[data_key] = js.loads(resp.content)
            else:
                return None
        else:
            return None
    
    def _call_url(self, method:str, url:str, kwargs:dict):
        for _ in range(self.url_retrys):
            resp = rq.request(method, url, **kwargs)
            if resp.status_code == 200: return resp
            sleep(0.01)
        
        return None

In [3]:
tracker = Tracker(144981811)

In [4]:
tracker.start_threads()

In [11]:
tracker._token._dict

{'db': 'NewEdenAnalytics',
 'expires': datetime.datetime(2019, 10, 5, 23, 50),
 'value': 'eyJhbGciOiJSUzI1NiIsImtpZCI6IkpXVC1TaWduYXR1cmUtS2V5IiwidHlwIjoiSldUIn0.eyJzY3AiOlsiZXNpLWxvY2F0aW9uLnJlYWRfbG9jYXRpb24udjEiLCJlc2ktbG9jYXRpb24ucmVhZF9zaGlwX3R5cGUudjEiLCJlc2ktYXNzZXRzLnJlYWRfYXNzZXRzLnYxIiwiZXNpLWZsZWV0cy5yZWFkX2ZsZWV0LnYxIiwiZXNpLWZsZWV0cy53cml0ZV9mbGVldC52MSIsImVzaS11aS53cml0ZV93YXlwb2ludC52MSIsImVzaS1sb2NhdGlvbi5yZWFkX29ubGluZS52MSJdLCJqdGkiOiJjMzM0ZjU3Mi1lODkzLTQ0NzUtYWRmNS1jNWU2NmYwYzQxNzAiLCJraWQiOiJKV1QtU2lnbmF0dXJlLUtleSIsInN1YiI6IkNIQVJBQ1RFUjpFVkU6MTQ0OTgxODExIiwiYXpwIjoiYjlmYWM1YmI3YzViNGZlODk3MjVlN2UzMWM0MDEyNjkiLCJuYW1lIjoiQ2FsdmluIiwib3duZXIiOiJpQU1Hc3A1TUVzTE5HNXo1aGRwcHhMQ0JoY1E9IiwiZXhwIjoxNTcwMzE5NDAyLCJpc3MiOiJsb2dpbi5ldmVvbmxpbmUuY29tIn0.nSveUwGaI4KPHcfTMq_QBOTXGq-RWAOymm6rwkGVo7tuapFTtA3V-4TDCpGpEy_mVPsMxSim2_m-KpYm5pkRO3Rm70ihGP1H-LIuf8zXubCsTZVRtHvEifvImJSldizvl6vD0-0E29Pa46bRz_wHgvdrUstX-lCyOXlYf1tZ_yZml2Y7yieDpzFcLR0hqEnhW0HLxDPnjo8J_S-PUvf_n_QgfH2eaY8klA

In [12]:
tracker._data._dict

{'ship': {'ship_item_id': 1031377964730,
  'ship_name': 'Lard Sniffer',
  'ship_type_id': 17480},
 'location': {'solar_system_id': 30004125, 'structure_id': 1027327111023},
 'online': {'last_login': '2019-10-05T21:18:12Z',
  'last_logout': '2019-10-05T23:35:47Z',
  'logins': 6384,
  'online': False}}

In [13]:
tracker._threads._dict

{'data': [<Thread(Thread-5, started daemon 140474587313920)>,
  <Thread(Thread-6, started daemon 140474578921216)>,
  <Thread(Thread-7, started daemon 140474570528512)>],
 'token': <Thread(Thread-4, started daemon 140474595706624)>}